# Splits extraction using dedicated service - F#

### Overview
This sample requests **on-demand** splits by calling a dedicated data service.

### Inputs/outputs
Splits sample is designed to help you searching and requesting dividends for a given instrument.
* It takes instrument specification parameter(s) as per input filter and returns *Dividends*

### Services used
This sample uses *gRPC requests* in order to retrieve splits data from the hosted service. The queried endpoint in this script are:
* *SplitsService*: to directly retrieve splits data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.CorporateActions.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run splits sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.CorporateActions.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Retrieve splits

#### 3.1 Equity selection

In [ ]:
// Set the instrument identifier 
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 Create and process request

In [ ]:
// instantiate the splits service
let service = new SplitsService.SplitsServiceClient(channel)

// Generate the splits request
let request = 
    new SplitsRequest (
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker )
    )

// Get the splits reply
let reply = service.Splits(request, headers)

#### 3.3 Retrieve splits

In [ ]:
// Visualize splits results
reply.Data
|> display

#### 3.4 Add dates constraints

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date(Year = 2020, Month = 01, Day = 01),
    EndDate = new Date(Year = 2021, Month = 03, Day = 31)
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);

In [ ]:
// Generate the splits request with dates constraints
let request = 
    new SplitsRequest( 
        Identifier = new Identifier( Exchange = exchange, Ticker = ticker),
        Constraints = constraints
    )

// Get the splits reply with dates constraints
let reply = service.Splits(request, headers)

#### 3.5 Retrieve splits

In [ ]:
// Visualize splits results
reply.Data
|> display